In [0]:
import requests
from pyspark.sql import Row
import json

def get_cat(API_KEY):
    # url = dbutils.secrets.get("SecretsStorm", "URL_TOKEN_STORM")
    url = f"https://api.thecatapi.com/v1/images/search?api_key={API_KEY}"

    headers = { 'x-api-key' : f'{API_KEY}' }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return print(f"Error: {response.status_code}, {response.text}")

In [0]:
API_KEY = dbutils.secrets.get("SecretsApiTest", "API_KEY")

data = get_cat(API_KEY)

In [0]:
import time

all_data = []
page = 1

while True and page <= 10:
    data = get_cat(API_KEY)
    time.sleep(1)
    all_data.extend(data)
    page += 1
    print(f'Número da página: {page}')

    time.sleep(1)

Número da página: 2
Número da página: 3
Número da página: 4
Número da página: 5
Número da página: 6
Número da página: 7
Número da página: 8
Número da página: 9
Número da página: 10
Número da página: 11


In [0]:
df = spark.read.json(spark.sparkContext.parallelize([all_data]))

In [0]:
df.createOrReplaceTempView('tempTbCat')

In [0]:
dfStg = spark.sql("""SELECT *,
                  dateadd(hour,-3,current_timestamp()) as dt_Creation
                  from tempTbCat                 
""")

In [0]:
(dfStg
    .write
    .format('delta')
    .mode('overWrite')
    .save('/mnt/DATALAKE/dev/Api_Test/tbApiCat/stage')
)
spark.sql("create table if not exists dev.tb_stg_api_cat using delta location '/mnt/DATALAKE/dev/Api_Test/tbApiCat/stage'")

DataFrame[]